In [19]:
import os
import speech_recognition as sr
from tqdm import tqdm
from multiprocessing.dummy import Pool

In [20]:
pool = Pool(8) # Number of concurrent threads

In [3]:
# with open("api-key.json") as f:
#     #GOOGLE_CLOUD_SPEECH_CREDENTIALS = f.read()
#     s = r.recognize_google(audio, language="en-US")

In [27]:
# https://github.com/akras14/speech-to-text
r = sr.Recognizer()
files = sorted(os.listdir('parts'))
files

['out000000000.wav',
 'out000000001.wav',
 'out000000003.wav',
 'out000000006.wav']

In [28]:
def transcribe(data):
    idx, file = data
    name = "parts/" + file
    print(name + " started")
    # Load audio file
    with sr.AudioFile(name) as source:
        audio = r.record(source)
    # Transcribe audio file
    #text = r.recognize_google_cloud(audio, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS)
    text  = r.recognize_google(audio, language="en-US")
    print(name + " done")
    return {
        "idx": idx,
        "text": text
    }

In [29]:
all_text = pool.map(transcribe, enumerate(files))
pool.close()
pool.join()

parts/out000000000.wav startedparts/out000000001.wav startedparts/out000000003.wav startedparts/out000000006.wav started



parts/out000000000.wav done
parts/out000000003.wav done
parts/out000000006.wav done
parts/out000000001.wav done


In [30]:
transcript = ""
for t in sorted(all_text, key=lambda x: x['idx']):
    total_seconds = t['idx'] * 30
    # Cool shortcut from:
    # https://stackoverflow.com/questions/775049/python-time-seconds-to-hms
    # to get hours, minutes and seconds
    m, s = divmod(total_seconds, 60)
    h, m = divmod(m, 60)

    # Format time as h:m:s - 30 seconds of text
    transcript = transcript + "{:0>2d}:{:0>2d}:{:0>2d} {}\n".format(h, m, s, t['text'])

print(transcript)

00:00:00 settle down
00:00:30 geode
00:01:00 you know how it is
00:01:30 would have no no nevermind



In [ ]:
with open("transcript.txt", "w") as f:
    f.write(transcript)